In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
from subprocess import check_output
print(check_output(["ls", "../kaggle/input"]).decode("utf8"))

sample_submission.csv
test.csv
train.csv



In [4]:
# Any results you write to the current directory are saved as output.

from sklearn import ensemble, metrics, linear_model
import random

#Some parameters to play with
rnd=12
random.seed(rnd)
n_ft=15 #Number of features to add
max_elts=5 #Maximum size of a group of linear features

In [5]:
class addNearestNeighbourLinearFeatures:
    
    def __init__(self, n_neighbours=1, max_elts=None, verbose=True, random_state=None):
        self.rnd=random_state
        self.n=n_neighbours
        self.max_elts=max_elts
        self.verbose=verbose
        self.neighbours=[]
        self.clfs=[]
        
    def fit(self,train,y):
        if self.rnd!=None:
            random.seed(rnd)
        if self.max_elts==None:
            self.max_elts=len(train.columns)
        list_vars=list(train.columns)
        random.shuffle(list_vars)
        
        lastscores=np.zeros(self.n)+1e15

        for elt in list_vars[:self.n]:
            self.neighbours.append([elt])
        list_vars=list_vars[self.n:]
        
        for elt in list_vars:
            indice=0
            scores=[]
            for elt2 in self.neighbours:
                if len(elt2)<self.max_elts:
                    clf=linear_model.LinearRegression(fit_intercept=False, normalize=True, copy_X=True, n_jobs=-1) 
                    clf.fit(train[elt2+[elt]], y)
                    scores.append(metrics.log_loss(y,clf.predict(train[elt2 + [elt]])))
                    indice=indice+1
                else:
                    scores.append(lastscores[indice])
                    indice=indice+1
            gains=lastscores-scores
            if gains.max()>0:
                temp=gains.argmax()
                lastscores[temp]=scores[temp]
                self.neighbours[temp].append(elt)

        indice=0
        for elt in self.neighbours:
            clf=linear_model.LinearRegression(fit_intercept=False, normalize=True, copy_X=True, n_jobs=-1) 
            clf.fit(train[elt], y)
            self.clfs.append(clf)
            if self.verbose:
                print(indice, lastscores[indice], elt)
            indice=indice+1
                    
    def transform(self, train):
        indice=0
        for elt in self.neighbours:
            train['_'.join(pd.Series(elt).sort_values().values)]=self.clfs[indice].predict(train[elt])
            indice=indice+1
        return train
    
    def fit_transform(self, train, y):
        self.fit(train, y)
        return self.transform(train)

In [6]:
train = pd.read_csv("../kaggle/input/train.csv")
target = train['target'].values
test = pd.read_csv("../kaggle/input/test.csv")
id_test = test['ID'].values

In [7]:
train['v22-1']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[0]))
test['v22-1']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[0]))
train['v22-2']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[1]))
test['v22-2']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[1]))
train['v22-3']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[2]))
test['v22-3']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[2]))
train['v22-4']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[3]))
test['v22-4']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[3]))

In [8]:
drop_list=['v91','v1', 'v8', 'v10', 'v15', 'v17', 'v25', 'v29', 'v34', 'v41', 'v46', 'v54', 'v64', 'v67', 'v97', 'v105', 'v111', 'v122']
train = train.drop(['ID','target'] + drop_list,axis=1).fillna(-999)
test = test.drop(['ID'] + drop_list,axis=1).fillna(-999)

In [9]:
refcols=list(train.columns)

In [10]:
for elt in refcols:
    if train[elt].dtype=='O':
        train[elt], temp = pd.factorize(train[elt])
        test[elt]=temp.get_indexer(test[elt])
    else:
        train[elt]=train[elt].round(5)
        test[elt]=test[elt].round(5)

In [11]:
a=addNearestNeighbourLinearFeatures(n_neighbours=n_ft, max_elts=max_elts, verbose=True, random_state=rnd)
a.fit(train, target)

(0, 0.57782344665658947, ['v32', 'v68', 'v37', 'v117', 'v63'])
(1, 0.72496249783039002, ['v36', 'v9', 'v60', 'v114', 'v128'])
(2, 1.9731142284859129, ['v5', 'v2', 'v71', 'v44', 'v75'])
(3, 0.57846574281780161, ['v123', 'v66', 'v23', 'v107', 'v22-2'])
(4, 0.60539821964227858, ['v87', 'v33', 'v120', 'v14', 'v40'])
(5, 0.97389758914091118, ['v93', 'v130', 'v72', 'v116', 'v88'])
(6, 1.2074071837133578, ['v126', 'v127', 'v24', 'v112', 'v104'])
(7, 0.56168242823017167, ['v102', 'v62', 'v22-4', 'v55', 'v83'])
(8, 0.57226864204831296, ['v53', 'v27', 'v80', 'v121', 'v98'])
(9, 0.88987314602797896, ['v124', 'v28', 'v82', 'v18', 'v26'])
(10, 0.59457273731407845, ['v51', 'v43', 'v65', 'v45', 'v70'])
(11, 0.74703780843945022, ['v76', 'v101', 'v74', 'v79', 'v59'])
(12, 0.67172294431249535, ['v48', 'v92', 'v110', 'v38', 'v20'])
(13, 1.7511631360700857, ['v125', 'v50', 'v85', 'v119', 'v113'])
(14, 0.94740968110166568, ['v94', 'v61', 'v108', 'v86', 'v30'])


In [12]:
train = a.transform(train)
test = a.transform(test)

In [13]:
clf = ensemble.ExtraTreesClassifier(n_estimators=750,max_features=55,criterion= 'entropy',min_samples_split= 4,
                        max_depth= 35, min_samples_leaf= 2, n_jobs = -1, random_state=rnd)

In [14]:
clf.fit(train,target)
pred_et=clf.predict_proba(test)

In [15]:
submission=pd.read_csv('../kaggle/input/sample_submission.csv')
submission.index=submission.ID
submission.PredictedProb=pred_et[:,1]
submission.to_csv('./addNNLinearFt.csv', index=False)
submission.PredictedProb.hist(bins=30)